In [13]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_community.graphs import Neo4jGraph
import os

from langchain.chat_models import ChatOpenAI

api_key =os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(openai_api_key=api_key)
Neo4jGraph_client = Neo4jGraph(
            url=os.getenv('NEO4J_URI'), username=os.getenv('NEO4J_USER'), password=os.getenv('NEO4J_PASSWORD')
        )
query = "MATCH (n) RETURN n"
result = Neo4jGraph_client.query(query)
def extract_names_from_result(result):
    names = []
    for item in result:
        # Check if 'name' key exists and add its value to the names list
        if 'name' in item['n']:
            names.append(item['n']['name'])
    return names

# Assuming 'result' is the output you've shown
names_list = extract_names_from_result(result)

system_template="Find the cloest word match in the {database_nodes} and replace it, and give me the correct user input"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

user_input = "Which users have access to the Executive Management Databasee and what are their roles?"
human_template="Replace the words in: {user_input} with the correct words in database_nodes."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_template, human_message_prompt])    
                                                           # get a chat completion from the formatted messages
request = chat_prompt.format_prompt(user_input=user_input, database_nodes=names_list).to_messages()


result = chat(request)
print(result.content)

Which users have access to the Executive_Management_Database and what are their roles?
